# Profiling and Optimization

In [28]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import dask
import dask_mongo
import dask.dataframe as dd

from langcodes import Language

In [2]:
sns.set_theme(style='darkgrid')

## Connecting to MongoDB

In [30]:
from dask.diagnostics import ProgressBar, Profiler

In [5]:
%%time

b = dask_mongo.read_mongo(
    database='ed23db',
    collection='users',
    chunksize=200_000,
    connection_kwargs={
        'host': 'localhost',
        'port': 27017
    }
)

CPU times: user 304 ms, sys: 91.5 ms, total: 396 ms
Wall time: 13min 35s


In [134]:
df = b.to_dataframe(meta={
#     '_id': str,
    'userid': 'str',
    'username': 'str',
    'location': 'str',
    'following': 'int32',
    'followers': 'int32',
    'totaltweets': 'int32',
    'usercreatedts': 'str'
#     'usercreatedts': 'datetime64[s]'
})

In [135]:
df.columns

Index(['userid', 'username', 'location', 'following', 'followers',
       'totaltweets', 'usercreatedts'],
      dtype='object')

In [136]:
df.dtypes

userid           object
username         object
location         object
following         int32
followers         int32
totaltweets       int32
usercreatedts    object
dtype: object

In [137]:
df['usercreatedts'] = df['usercreatedts'].map(lambda dt: dt[:19])

In [138]:
df['usercreatedts'] = dd.to_datetime(df['usercreatedts'], format='%Y-%m-%d %H:%M:%S')

In [139]:
df.dtypes

userid                   object
username                 object
location                 object
following                 int32
followers                 int32
totaltweets               int32
usercreatedts    datetime64[ns]
dtype: object

In [140]:
df.head(20)

,userid,username,location,following,followers,totaltweets,usercreatedts
0,1237027988287471618,pbi_es,Palestina,86,6855,6401,2020-03-09 14:50:40
1,1407899290790473732,SectionChine,,1718,1668,335,2021-06-24 06:06:32
2,6509832,CNNnews18,India,399,4730654,997125,2007-06-01 20:31:01
3,1570222426570608641,AmazingMeta,Los Angeles,86,26,276,2022-09-15 01:26:48
4,20918680,CallMeAdamNYC,"New York, NY",2228,2479,13155,2009-02-15 16:24:01
5,216893584,lavisionatl,Atlanta Georgia,942,3124,52076,2010-11-18 00:38:14
6,62089976,yunec,"Kraków, Polska",2582,6560,132814,2009-08-01 18:35:04
7,209520717,olivanoticias,México,381,25068,249266,2010-10-29 10:39:49
8,1573729011364241409,funny0animals,,1,17,307,2022-09-24 17:40:19
9,916681683134447617,MilaPlayporn,,19,8917,51528,2017-10-07 15:08:39


## Profiling simple operation

### MongoDB

In [21]:
%%time

with ProgressBar(dt=2.0), Profiler() as prof2:
    df.shape[0].compute(), df.shape[1]  # all entries

[########################################] | 100% Completed | 14m 22s
CPU times: user 14min 30s, sys: 26.7 s, total: 14min 56s
Wall time: 14min 22s


In [41]:
prof2.visualize()

figure(id='p3214', ...)

In [43]:
!mv profile.html profile-mongo-shape.html

### Parquet

In [54]:
import pyarrow as pa

In [141]:
%%time

df.to_parquet(
    '../parquet/users.parquet',
    engine='pyarrow',
    compression=None,
    overwrite=True,
#     schema={
#         'userid': pa.string(),
#         'username': pa.string(),
#         'location': pa.string(),
#         'following': pa.int32(),
#         'followers': pa.int32(),
#         'totaltweets': pa.int32(),
#         'usercreatedts': pa.date64()
#     }
)

CPU times: user 21min 55s, sys: 1min 6s, total: 23min 1s
Wall time: 20min 27s


In [142]:
dfp = dd.read_parquet('../parquet/users.parquet', engine='pyarrow')

In [143]:
%%time

with ProgressBar(dt=2.0), Profiler() as prof3:
    dfp.shape[0].compute(), df.shape[1]  # all entries

[########################################] | 100% Completed | 65.83 s
CPU times: user 1min 14s, sys: 15.7 s, total: 1min 29s
Wall time: 1min 5s


In [144]:
prof3.visualize()

figure(id='p3510', ...)

In [145]:
!mv profile.html profile-parquet-shape.html

14x times faster!

In [147]:
%%time

with ProgressBar(dt=2.0):
    print(dfp['userid'].unique().shape[0].compute())  # unique user ids

[########################################] | 100% Completed | 74.63 s
6538298
CPU times: user 1min 51s, sys: 14 s, total: 2min 5s
Wall time: 1min 14s


In [148]:
%%time

with ProgressBar(dt=2.0):
    followers_distribution = dfp \
        .groupby('userid', group_keys=True, sort=False)['followers'] \
        .max() \
        .value_counts() \
        .compute()

[########################################] | 100% Completed | 202.28 s
CPU times: user 4min 17s, sys: 18.2 s, total: 4min 35s
Wall time: 3min 22s


In [149]:
followers_distribution

followers
0           320261
1           171917
2           125541
3           101097
4            87141
             ...  
61572            1
61573            1
61574            1
61577            1
54997992         1
Name: count, Length: 73339, dtype: int64

In [171]:
dfp.loc[[2, 3]]

KeyError: 'Cannot index with list against unknown division. Try setting divisions using ``ddf.set_index``'

In [170]:
dfp.memory_usage().compute()

Index            1058099408
followers         529049704
following         529049704
location         1058099408
totaltweets       529049704
usercreatedts    1058099408
userid           1058099408
username         1058099408
dtype: int64

In [173]:
dfp.loc[dfp['followers'] < 10].head(10)

,userid,username,location,following,followers,totaltweets,usercreatedts
14,1579614019765223424,KatKai19,mexico?,1,1,2,2022-10-10 23:25:14
18,1452636766578429960,desi_sattayar,,25,8,160,2021-10-25 14:03:20
35,1575211040438210561,getthaoutmyway,Hollywoo,49,0,88,2022-09-28 19:50:37
42,1579458964382781440,mad_humanity,,12,0,12,2022-10-10 13:09:14
46,1578075076766973954,Costituzione48,🇮🇹,71,5,309,2022-10-06 17:30:14
53,1574443687316262917,JERRYNOIAA,,10,8,14,2022-09-26 17:00:20
72,1579618529166311424,LadyVPutin,,9,0,5,2022-10-10 23:43:55
82,1575271827445755905,Jesseorozcouae,"Dubai, United Arab Emirates",185,9,492,2022-09-28 23:51:06
85,1568019204422377477,sovabravo,"Reston, VA",36,1,26,2022-09-08 23:31:33
92,1177696950608650240,FlyinRy32764747,,0,0,150,2019-09-27 21:30:14


In [153]:
with ProgressBar(dt=2.0):
    unique_userids = dfp['userid'].unique().compute()

[########################################] | 100% Completed | 72.63 s


In [157]:
with open('../parquet/unique_userids.txt', 'w', encoding='utf-8') as f:
    for userid in unique_userids.values:
        f.write(userid + '\n')